In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from mlxtend.classifier import StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split
from imblearn.metrics import geometric_mean_score, sensitivity_score, specificity_score
from xgboost import XGBClassifier
from shaphypetune import BoostRFE
from sklearn.base import BaseEstimator
from sklearn.utils.metaestimators import if_delegate_has_method

import torch
import re, pickle, random, os
import warnings
warnings.filterwarnings('ignore')

from collections import Counter

In [2]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

RANDOM_STATE = 42
seed_everything(seed=RANDOM_STATE)

In [3]:
col_names = ['Sequence Name','mcg', 'gvh', 'alm', 'mit', 'erl','pox','vac','nuc','class']
df = pd.read_csv("yeast.csv", names = col_names, delim_whitespace=True)
df.shape

(1484, 10)

In [4]:
df.head()

,Sequence Name,mcg,gvh,alm,mit,erl,pox,vac,nuc,class
0,ADT1_YEAST,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,ADT2_YEAST,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,ADT3_YEAST,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,AAR2_YEAST,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,AATM_YEAST,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT


In [5]:
del df['Sequence Name']
df.head()

,mcg,gvh,alm,mit,erl,pox,vac,nuc,class
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT


In [6]:
df['class'].value_counts()

CYT    463
NUC    429
MIT    244
ME3    163
ME2     51
ME1     44
EXC     35
VAC     30
POX     20
ERL      5
Name: class, dtype: int64

In [7]:
df.isnull().sum()

mcg      0
gvh      0
alm      0
mit      0
erl      0
pox      0
vac      0
nuc      0
class    0
dtype: int64

In [8]:
df.describe()

,mcg,gvh,alm,mit,erl,pox,vac,nuc
count,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000
mean,0.500121,0.499933,0.500034,0.261186,0.504717,0.007500,0.499885,0.276199
std,0.137299,0.123924,0.086670,0.137098,0.048351,0.075683,0.057797,0.106491
min,0.110000,0.130000,0.210000,0.000000,0.500000,0.000000,0.000000,0.000000
25%,0.410000,0.420000,0.460000,0.170000,0.500000,0.000000,0.480000,0.220000
50%,0.490000,0.490000,0.510000,0.220000,0.500000,0.000000,0.510000,0.220000
75%,0.580000,0.570000,0.550000,0.320000,0.500000,0.000000,0.530000,0.300000
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.830000,0.730000,1.000000


# Train and Test Split

In [9]:
x = df.drop(['class'], axis=1)
y = df['class']

X_train,X_test,y_train,y_test=train_test_split(x, y, train_size=0.8, stratify = y, random_state=100)

In [10]:
y_train.shape, y_test.shape

((1187,), (297,))

In [11]:
y_train.value_counts()/len(y_train)

CYT    0.311710
NUC    0.288964
MIT    0.164280
ME3    0.110362
ME2    0.034541
ME1    0.029486
EXC    0.023589
VAC    0.020219
POX    0.013479
ERL    0.003370
Name: class, dtype: float64

In [12]:
y_test.value_counts()/len(y_test)

CYT    0.313131
NUC    0.289562
MIT    0.164983
ME3    0.107744
ME2    0.033670
ME1    0.030303
EXC    0.023569
VAC    0.020202
POX    0.013468
ERL    0.003367
Name: class, dtype: float64

In [13]:
from sklearn.preprocessing import StandardScaler
Scaler_X = StandardScaler()
X_train = Scaler_X.fit_transform(X_train)
X_test = Scaler_X.transform(X_test)

# Model Building

In [14]:
model = list()
precision = list()
recall = list()
F1score = list()
AUCROC = list()
balanced_acc = list()
GMean = list()
sensi = list()
speci = list()

In [15]:
def test_eval(clf_model, X_test, y_test, algo=None):

    y_prob=clf_model.predict_proba(X_test)
    y_pred=clf_model.predict(X_test)

    print('Confusion Matrix')
    print('='*60)
    print(confusion_matrix(y_test,y_pred),"\n")
    print('Classification Report')
    print('='*60)
    print(classification_report(y_test,y_pred),"\n")
    print('='*60)
    print('AUC-ROC')
    print(roc_auc_score(y_test, y_prob, multi_class='ovr'), "\n")
    print('Balanced Accuracy')
    print(balanced_accuracy_score(y_test, y_pred),"\n")
    print('Geometric Mean Score')
    print(geometric_mean_score(y_test, y_pred, average='macro'),"\n")
    print('Sensitivity')
    print(sensitivity_score(y_test, y_pred, average='macro'),"\n")
    print('Specificity')
    print(specificity_score(y_test, y_pred, average='macro'))
          
    model.append(algo)
    precision.append(precision_score(y_test,y_pred, average='macro'))
    recall.append(recall_score(y_test,y_pred, average='macro'))
    F1score.append(f1_score(y_test,y_pred, average='macro'))
    AUCROC.append(roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro'))
    balanced_acc.append(balanced_accuracy_score(y_test, y_pred))
    GMean.append(geometric_mean_score(y_test, y_pred, average='macro'))
    sensi.append(sensitivity_score(y_test, y_pred, average='macro'))
    speci.append(specificity_score(y_test, y_pred, average='macro'))


## Model-1: Logistic Regression

In [16]:
clf_LR = LogisticRegression(multi_class='ovr', solver='liblinear')
clf_LR.fit(X_train, y_train)

LogisticRegression(multi_class='ovr', solver='liblinear')

In [17]:
test_eval(clf_LR, X_test, y_test, 'Logistic Regression')

Confusion Matrix
[[61  0  0  0  0  1  6 24  1  0]
 [ 0  1  0  0  0  0  0  0  0  0]
 [ 1  0  4  0  1  0  1  0  0  0]
 [ 1  0  1  5  0  1  1  0  0  0]
 [ 1  0  1  0  5  2  1  0  0  0]
 [ 4  0  0  0  0 27  0  1  0  0]
 [14  0  0  1  0  2 28  3  1  0]
 [40  1  0  0  0  2  9 34  0  0]
 [ 2  0  0  0  0  0  0  0  2  0]
 [ 1  0  1  0  0  3  0  1  0  0]] 

Classification Report
              precision    recall  f1-score   support

         CYT       0.49      0.66      0.56        93
         ERL       0.50      1.00      0.67         1
         EXC       0.57      0.57      0.57         7
         ME1       0.83      0.56      0.67         9
         ME2       0.83      0.50      0.62        10
         ME3       0.71      0.84      0.77        32
         MIT       0.61      0.57      0.59        49
         NUC       0.54      0.40      0.46        86
         POX       0.50      0.50      0.50         4
         VAC       0.00      0.00      0.00         6

    accuracy                    

## Model-2: Decision Tree

In [18]:
clf_DT = DecisionTreeClassifier()
clf_DT.fit(X_train, y_train)

DecisionTreeClassifier()

In [19]:
test_eval(clf_DT, X_test, y_test, 'Decision Tree')

Confusion Matrix
[[48  0  1  0  3  2  9 29  1  0]
 [ 0  0  0  0  1  0  0  0  0  0]
 [ 0  0  3  0  2  0  1  0  0  1]
 [ 0  0  0  5  1  0  1  0  1  1]
 [ 1  0  0  0  6  0  1  1  1  0]
 [ 1  0  0  0  2 19  3  5  0  2]
 [ 8  0  0  1  2  1 25 10  2  0]
 [29  0  0  0  1  6  8 40  0  2]
 [ 1  0  0  1  0  0  0  1  1  0]
 [ 1  1  0  0  1  2  1  0  0  0]] 

Classification Report
              precision    recall  f1-score   support

         CYT       0.54      0.52      0.53        93
         ERL       0.00      0.00      0.00         1
         EXC       0.75      0.43      0.55         7
         ME1       0.71      0.56      0.63         9
         ME2       0.32      0.60      0.41        10
         ME3       0.63      0.59      0.61        32
         MIT       0.51      0.51      0.51        49
         NUC       0.47      0.47      0.47        86
         POX       0.17      0.25      0.20         4
         VAC       0.00      0.00      0.00         6

    accuracy                    

## Model-3: Gaussian NB

In [20]:
clf_NB = GaussianNB()
clf_NB.fit(X_train, y_train)

GaussianNB()

In [21]:
test_eval(clf_NB, X_test, y_test, 'Gaussian NB')

Confusion Matrix
[[ 0  0  2  0  5 22  0  1  1 62]
 [ 0  1  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  1  0  0  0  0  1]
 [ 0  0  2  6  1  0  0  0  0  0]
 [ 0  1  0  1  8  0  0  0  0  0]
 [ 0  1  0  0  2 28  0  0  0  1]
 [ 0  0  2  2 22  4  2  1  1 15]
 [ 1  1  3  0  6 19  0  9  0 47]
 [ 0  0  1  0  0  0  0  0  2  1]
 [ 0  0  1  0  2  2  0  0  0  1]] 

Classification Report
              precision    recall  f1-score   support

         CYT       0.00      0.00      0.00        93
         ERL       0.25      1.00      0.40         1
         EXC       0.31      0.71      0.43         7
         ME1       0.67      0.67      0.67         9
         ME2       0.17      0.80      0.28        10
         ME3       0.37      0.88      0.52        32
         MIT       1.00      0.04      0.08        49
         NUC       0.82      0.10      0.19        86
         POX       0.50      0.50      0.50         4
         VAC       0.01      0.17      0.01         6

    accuracy                    

## Model-4: K-Nearest Neighbour

In [22]:
clf_KNN = KNeighborsClassifier()
clf_KNN.fit(X_train, y_train)

KNeighborsClassifier()

In [23]:
test_eval(clf_KNN, X_test, y_test, 'KNN')

Confusion Matrix
[[57  0  0  0  1  2  6 26  1  0]
 [ 0  1  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  1  0  0  1  0  0]
 [ 1  0  1  7  0  0  0  0  0  0]
 [ 2  0  2  2  3  0  1  0  0  0]
 [ 3  0  0  1  1 24  1  2  0  0]
 [12  0  1  1  0  2 28  4  1  0]
 [38  0  0  0  0  3  9 35  0  1]
 [ 1  0  0  1  0  0  0  0  2  0]
 [ 1  0  1  0  0  2  1  1  0  0]] 

Classification Report
              precision    recall  f1-score   support

         CYT       0.50      0.61      0.55        93
         ERL       1.00      1.00      1.00         1
         EXC       0.50      0.71      0.59         7
         ME1       0.58      0.78      0.67         9
         ME2       0.50      0.30      0.37        10
         ME3       0.73      0.75      0.74        32
         MIT       0.61      0.57      0.59        49
         NUC       0.51      0.41      0.45        86
         POX       0.50      0.50      0.50         4
         VAC       0.00      0.00      0.00         6

    accuracy                    

## MODEL-5 Support Vector Classifier

In [24]:
clf_SVC = SVC(probability=True)
clf_SVC.fit(X_train, y_train)

SVC(probability=True)

In [25]:
test_eval(clf_SVC, X_test, y_test, 'SVC')

Confusion Matrix
[[60  0  0  0  0  0  6 26  1  0]
 [ 0  0  0  0  0  0  0  1  0  0]
 [ 1  0  5  0  1  0  0  0  0  0]
 [ 0  0  1  7  0  1  0  0  0  0]
 [ 1  0  0  1  4  3  1  0  0  0]
 [ 3  0  0  1  0 26  1  1  0  0]
 [16  0  0  1  1  2 26  2  1  0]
 [36  0  0  0  0  2  8 40  0  0]
 [ 1  0  0  0  1  0  0  0  2  0]
 [ 1  0  1  0  0  2  1  1  0  0]] 

Classification Report
              precision    recall  f1-score   support

         CYT       0.50      0.65      0.57        93
         ERL       0.00      0.00      0.00         1
         EXC       0.71      0.71      0.71         7
         ME1       0.70      0.78      0.74         9
         ME2       0.57      0.40      0.47        10
         ME3       0.72      0.81      0.76        32
         MIT       0.60      0.53      0.57        49
         NUC       0.56      0.47      0.51        86
         POX       0.50      0.50      0.50         4
         VAC       0.00      0.00      0.00         6

    accuracy                    

# MODEL-6 XGBoost

In [26]:
class BoostRFEWrap(BaseEstimator, BoostRFE):

    @if_delegate_has_method(delegate='estimator')
    def predict_proba(self, X):
        return self.predict(X, method='predict_proba')


xgb_params = {'max_depth': (5,10),
          'learning_rate': (0.001, 0.3),
          'n_estimators': (25, 200),
          'reg_alpha' : (0.1, 1),
          'reg_lambda': (0.1, 1),
          'subsample': (0.5,  0.9),
          'colsample_bytree': (0.5,1),
          'min_child_weight': (0,10),
          'gamma': (0,1)}

clf_XGB = BoostRFEWrap(XGBClassifier(), param_grid=xgb_params, importance_type='shap_importances', train_importance=False, min_features_to_select=1, step=1)

In [27]:
xclf = clf_XGB.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=6, verbose=0)


512 trials detected for ('max_depth', 'learning_rate', 'n_estimators', 'reg_alpha', 'reg_lambda', 'subsample', 'colsample_bytree', 'min_child_weight', 'gamma')

trial: 0001 ### iterations: 00012 ### eval_score: 0.44444
trial: 0002 ### iterations: 00013 ### eval_score: 0.45454
trial: 0003 ### iterations: 00015 ### eval_score: 0.42424
trial: 0004 ### iterations: 00025 ### eval_score: 0.41077
trial: 0005 ### iterations: 00008 ### eval_score: 0.39057
trial: 0006 ### iterations: 00012 ### eval_score: 0.42424
trial: 0007 ### iterations: 00009 ### eval_score: 0.43434
trial: 0008 ### iterations: 00014 ### eval_score: 0.42761
trial: 0009 ### iterations: 00010 ### eval_score: 0.44781
trial: 0010 ### iterations: 00009 ### eval_score: 0.42424
trial: 0011 ### iterations: 00009 ### eval_score: 0.42424
trial: 0012 ### iterations: 00010 ### eval_score: 0.42424
trial: 0013 ### iterations: 00011 ### eval_score: 0.41751
trial: 0014 ### iterations: 00011 ### eval_score: 0.40741
trial: 0015 ### iterations

trial: 0140 ### iterations: 00007 ### eval_score: 0.45454
trial: 0141 ### iterations: 00010 ### eval_score: 0.41751
trial: 0142 ### iterations: 00012 ### eval_score: 0.42087
trial: 0143 ### iterations: 00003 ### eval_score: 0.44444
trial: 0144 ### iterations: 00003 ### eval_score: 0.44781
trial: 0145 ### iterations: 00009 ### eval_score: 0.45454
trial: 0146 ### iterations: 00008 ### eval_score: 0.45118
trial: 0147 ### iterations: 00012 ### eval_score: 0.46465
trial: 0148 ### iterations: 00013 ### eval_score: 0.46128
trial: 0149 ### iterations: 00015 ### eval_score: 0.41414
trial: 0150 ### iterations: 00001 ### eval_score: 0.41414
trial: 0151 ### iterations: 00003 ### eval_score: 0.45454
trial: 0152 ### iterations: 00003 ### eval_score: 0.45454
trial: 0153 ### iterations: 00007 ### eval_score: 0.44444
trial: 0154 ### iterations: 00009 ### eval_score: 0.44444
trial: 0155 ### iterations: 00007 ### eval_score: 0.46465
trial: 0156 ### iterations: 00008 ### eval_score: 0.46465
trial: 0157 ##

trial: 0282 ### iterations: 00009 ### eval_score: 0.42424
trial: 0283 ### iterations: 00007 ### eval_score: 0.41751
trial: 0284 ### iterations: 00006 ### eval_score: 0.41077
trial: 0285 ### iterations: 00004 ### eval_score: 0.41751
trial: 0286 ### iterations: 00004 ### eval_score: 0.41751
trial: 0287 ### iterations: 00008 ### eval_score: 0.40741
trial: 0288 ### iterations: 00012 ### eval_score: 0.40404
trial: 0289 ### iterations: 00009 ### eval_score: 0.40741
trial: 0290 ### iterations: 00012 ### eval_score: 0.43434
trial: 0291 ### iterations: 00013 ### eval_score: 0.43434
trial: 0292 ### iterations: 00015 ### eval_score: 0.43098
trial: 0293 ### iterations: 00007 ### eval_score: 0.41414
trial: 0294 ### iterations: 00008 ### eval_score: 0.40741
trial: 0295 ### iterations: 00017 ### eval_score: 0.41751
trial: 0296 ### iterations: 00010 ### eval_score: 0.43434
trial: 0297 ### iterations: 00012 ### eval_score: 0.41077
trial: 0298 ### iterations: 00010 ### eval_score: 0.42761
trial: 0299 ##

trial: 0424 ### iterations: 00003 ### eval_score: 0.45454
trial: 0425 ### iterations: 00007 ### eval_score: 0.42761
trial: 0426 ### iterations: 00012 ### eval_score: 0.43098
trial: 0427 ### iterations: 00028 ### eval_score: 0.44444
trial: 0428 ### iterations: 00017 ### eval_score: 0.45118
trial: 0429 ### iterations: 00013 ### eval_score: 0.41414
trial: 0430 ### iterations: 00015 ### eval_score: 0.41414
trial: 0431 ### iterations: 00001 ### eval_score: 0.44781
trial: 0432 ### iterations: 00000 ### eval_score: 0.46128
trial: 0433 ### iterations: 00012 ### eval_score: 0.43771
trial: 0434 ### iterations: 00016 ### eval_score: 0.43434
trial: 0435 ### iterations: 00013 ### eval_score: 0.47475
trial: 0436 ### iterations: 00015 ### eval_score: 0.47138
trial: 0437 ### iterations: 00004 ### eval_score: 0.40404
trial: 0438 ### iterations: 00004 ### eval_score: 0.40741
trial: 0439 ### iterations: 00003 ### eval_score: 0.46801
trial: 0440 ### iterations: 00003 ### eval_score: 0.46465
trial: 0441 ##

In [28]:
test_eval(xclf, X_test, y_test, 'XGB')

Confusion Matrix
[[63  0  0  0  0  0  4 26  0  0]
 [ 0  0  0  0  1  0  0  0  0  0]
 [ 2  0  4  0  1  0  0  0  0  0]
 [ 0  0  2  6  0  0  0  0  0  1]
 [ 1  0  0  1  6  0  2  0  0  0]
 [ 2  0  0  0  1 27  2  0  0  0]
 [17  0  1  1  0  3 25  2  0  0]
 [25  0  0  0  1  3  6 51  0  0]
 [ 2  0  0  0  1  1  0  0  0  0]
 [ 2  0  0  0  0  2  1  1  0  0]] 

Classification Report
              precision    recall  f1-score   support

         CYT       0.55      0.68      0.61        93
         ERL       0.00      0.00      0.00         1
         EXC       0.57      0.57      0.57         7
         ME1       0.75      0.67      0.71         9
         ME2       0.55      0.60      0.57        10
         ME3       0.75      0.84      0.79        32
         MIT       0.62      0.51      0.56        49
         NUC       0.64      0.59      0.61        86
         POX       0.00      0.00      0.00         4
         VAC       0.00      0.00      0.00         6

    accuracy                    

# MODEL-7 Stacking

In [29]:
clf_DT = DecisionTreeClassifier()
clf_SVC = SVC(probability=True)
clf_KNN = KNeighborsClassifier()
clf_LR = LogisticRegression()

sclf = StackingClassifier(classifiers=[clf_DT, clf_SVC, clf_KNN], use_probas=True, average_probas=False, meta_classifier=clf_LR)

In [30]:
sclf.fit(X_train, y_train)
test_eval(sclf, X_test, y_test, 'Stacking')

Confusion Matrix
[[47  0  1  0  2  1  8 32  1  1]
 [ 0  0  0  0  1  0  0  0  0  0]
 [ 0  0  4  0  1  0  1  0  0  1]
 [ 0  0  1  5  2  0  0  0  0  1]
 [ 1  0  0  0  6  0  1  1  1  0]
 [ 1  0  0  0  1 17  3  7  0  3]
 [ 8  0  0  1  1  1 25 10  3  0]
 [26  0  0  0  1  5  8 43  0  3]
 [ 1  0  0  1  0  0  0  1  1  0]
 [ 1  0  1  0  1  2  1  0  0  0]] 

Classification Report
              precision    recall  f1-score   support

         CYT       0.55      0.51      0.53        93
         ERL       0.00      0.00      0.00         1
         EXC       0.57      0.57      0.57         7
         ME1       0.71      0.56      0.63         9
         ME2       0.38      0.60      0.46        10
         ME3       0.65      0.53      0.59        32
         MIT       0.53      0.51      0.52        49
         NUC       0.46      0.50      0.48        86
         POX       0.17      0.25      0.20         4
         VAC       0.00      0.00      0.00         6

    accuracy                    

# MODEL-8 Bagging

In [31]:
clf_bg = BaggingClassifier(clf_DT)

In [32]:
clf_bg.fit(X_train, y_train)
test_eval(clf_bg, X_test, y_test, 'Bagging')

Confusion Matrix
[[49  0  0  0  1  2  7 33  1  0]
 [ 0  0  0  0  1  0  0  0  0  0]
 [ 1  0  5  0  1  0  0  0  0  0]
 [ 0  0  1  5  1  0  1  0  0  1]
 [ 1  0  0  0  5  0  4  0  0  0]
 [ 4  0  0  0  1 22  3  2  0  0]
 [15  0  0  1  0  1 24  7  1  0]
 [28  0  0  0  1  6  4 47  0  0]
 [ 1  0  0  1  0  1  0  0  1  0]
 [ 2  0  1  0  1  2  0  0  0  0]] 

Classification Report
              precision    recall  f1-score   support

         CYT       0.49      0.53      0.51        93
         ERL       0.00      0.00      0.00         1
         EXC       0.71      0.71      0.71         7
         ME1       0.71      0.56      0.63         9
         ME2       0.42      0.50      0.45        10
         ME3       0.65      0.69      0.67        32
         MIT       0.56      0.49      0.52        49
         NUC       0.53      0.55      0.54        86
         POX       0.33      0.25      0.29         4
         VAC       0.00      0.00      0.00         6

    accuracy                    

In [33]:
clf_eval_df = pd.DataFrame({'model':model,
                            'precision':precision,
                            'recall':recall,
                            'f1-score':F1score,
                            'AUC-ROC':AUCROC,
                            'balanced_acc':balanced_acc,
                            'GMean':GMean,
                            'sensitivity':sensi,
                            'specificity':speci})
clf_eval_df

,model,precision,recall,f1-score,AUC-ROC,balanced_acc,GMean,sensitivity,specificity
0,Logistic Regression,0.558500,0.559343,0.540667,0.876383,0.559343,0.725387,0.559343,0.940724
1,Decision Tree,0.409472,0.391933,0.389994,0.663265,0.391933,0.605227,0.391933,0.934598
2,Gaussian NB,0.409871,0.486809,0.308444,0.802018,0.486809,0.668158,0.486809,0.917064
3,KNN,0.542220,0.563337,0.545753,0.835868,0.563337,0.727410,0.563337,0.939269
4,SVC,0.488017,0.484545,0.482723,0.874926,0.484545,0.675712,0.484545,0.942299
5,XGB,0.443201,0.446249,0.442781,0.772155,0.446249,0.650417,0.446249,0.947997
6,Stacking,0.402353,0.402381,0.397087,0.856613,0.402381,0.613348,0.402381,0.934922
7,Bagging,0.439701,0.427053,0.431025,0.774247,0.427053,0.632764,0.427053,0.937566


In [34]:
clf_eval_df.to_excel("D:\Skripsi\Final\project_yeast_final.xlsx")